In [17]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Window as W
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder.getOrCreate()

22/08/10 10:30:53 WARN Utils: Your hostname, francesco-TravelMate-P238-M resolves to a loopback address: 127.0.1.1; using 192.168.1.129 instead (on interface wlp3s0)
22/08/10 10:30:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/08/10 10:30:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.format('csv').option('header', 'true').load("owid-covid-data.csv")
df.select("iso_code", 'location', 'population', 'total_cases').show(3)



22/08/10 10:31:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-----------+----------+-----------+
|iso_code|   location|population|total_cases|
+--------+-----------+----------+-----------+
|     AFG|Afghanistan|38928341.0|        1.0|
|     AFG|Afghanistan|38928341.0|        1.0|
|     AFG|Afghanistan|38928341.0|        1.0|
+--------+-----------+----------+-----------+
only showing top 3 rows



In [4]:
# df.schema.json()
# 15 стран с наибольшим процентом переболевших на 31 марта
res_df = df.select("iso_code", 'location', 'population', 'new_cases', 'date')\
.filter(F.col("date") <= '2022-03-31' ).groupBy(['iso_code', 'location']).agg({'population': 'max', 'new_cases' : 'sum'})
res_df = res_df.withColumn('percent', F.col("sum(new_cases)") / F.col("max(population)") * 100)
res_df.sort('percent', ascending = False).show(15)

+--------+-------------+--------------+---------------+------------------+
|iso_code|     location|sum(new_cases)|max(population)|           percent|
+--------+-------------+--------------+---------------+------------------+
|     AND|      Andorra|       12771.0|        77265.0|16.528829353523587|
|     MNE|   Montenegro|       95377.0|       628062.0|15.185921135174551|
|     CZE|      Czechia|     1600341.0|     10708982.0|  14.9439134364032|
|     SMR|   San Marino|        5026.0|        33938.0|14.809358241499204|
|     SVN|     Slovenia|      231593.0|      2078932.0|11.139998807079788|
|     LUX|   Luxembourg|       64746.0|       625976.0|10.343208046314874|
|     SRB|       Serbia|      660299.0|      6804596.0| 9.703720838092373|
|     ISR|       Israel|      836883.0|      8655541.0| 9.668754385196719|
|     BHR|      Bahrain|      163113.0|      1701583.0| 9.585956136139112|
|     USA|United States|   3.1628012E7|    331002647.0| 9.555214221595032|
|     SWE|       Sweden| 

In [27]:
# 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
res_df = df.select("iso_code", 'new_cases', 'location','date')\
.filter(F.col("date") <= '2021-03-31' )\
    .filter(F.col("date") >= '2021-03-28' )\
        .filter(~F.col('iso_code').like('OWID_%') )\
            .groupBy(['iso_code', 'location',  'date'] ).agg({'new_cases' : 'max'})
# res_df = res_df.withColumn('percent', F.col("sum(total_cases)") / F.col("max(population)") )
res_df2 = res_df.withColumn("max(new_cases)", res_df["max(new_cases)"].cast(IntegerType())).sort('max(new_cases)', ascending = False)
res_df2.drop_duplicates(subset=['iso_code']).sort('max(new_cases)', ascending = False).show(10)



+--------+-------------+----------+--------------+
|iso_code|     location|      date|max(new_cases)|
+--------+-------------+----------+--------------+
|     BRA|       Brazil|2021-03-31|         90638|
|     IND|        India|2021-03-31|         72330|
|     USA|United States|2021-03-29|         69429|
|     FRA|       France|2021-03-31|         59054|
|     TUR|       Turkey|2021-03-31|         39302|
|     POL|       Poland|2021-03-31|         32891|
|     DEU|      Germany|2021-03-31|         25014|
|     ITA|        Italy|2021-03-31|         23887|
|     SWE|       Sweden|2021-03-30|         16427|
|     ARG|    Argentina|2021-03-31|         16056|
+--------+-------------+----------+--------------+
only showing top 10 rows



In [23]:
res_rus = df.filter(F.col('iso_code') == 'RUS').filter(F.col("date") <= '2021-03-31' )\
    .filter(F.col("date") >= '2021-03-28' ).select("iso_code", 'new_cases', 'date')
w = W.orderBy('date')
res_rus = res_rus.withColumn('yesterday_cases', F.lag('new_cases', 1).over(w))
res_rus = res_rus.withColumn('diff_cases', F.col('new_cases') - F.col('yesterday_cases') )
res_rus.filter(F.col('diff_cases').isNotNull()).show()


22/08/07 20:59:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+---------+----------+---------------+----------+
|iso_code|new_cases|      date|yesterday_cases|diff_cases|
+--------+---------+----------+---------------+----------+
|     RUS|   8589.0|2021-03-29|         8979.0|    -390.0|
|     RUS|   8162.0|2021-03-30|         8589.0|    -427.0|
|     RUS|   8156.0|2021-03-31|         8162.0|      -6.0|
+--------+---------+----------+---------------+----------+



22/08/07 20:59:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+---------+----------+---------------+----------+
|iso_code|new_cases|      date|yesterday_cases|diff_cases|
+--------+---------+----------+---------------+----------+
|     RUS|   8589.0|2021-03-29|         8979.0|    -390.0|
|     RUS|   8162.0|2021-03-30|         8589.0|    -427.0|
|     RUS|   8156.0|2021-03-31|         8162.0|      -6.0|
+--------+---------+----------+---------------+----------+

